# <a name="setup_section"></a> 1. Setup: Manage Installations Imports and Helper Functions


In [3]:
import numpy as np
from tqdm import tqdm

In [5]:
np.arange(10,20)

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [12]:
for i in enumerate(tqdm(np.arange(10, 20))):
    print(i)

  1%|▊                                                                             | 10/1000 [00:00<00:00, 9946.18it/s]

(0, 10)
(1, 11)
(2, 12)
(3, 13)
(4, 14)
(5, 15)
(6, 16)
(7, 17)
(8, 18)
(9, 19)


In [1]:
#!pip install --upgrade GPy
#!pip install --upgrade numpy

In [2]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import csv
import GPy
import numpy as np

from datetime import datetime
from glob import glob
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm, percentileofscore

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


# <a name="loading_datasets"></a> 2. Loading Datasets


## Load metadata


In [3]:
metdata_path = "..\\data\\metadata\\"
metadata = pd.read_csv(metdata_path + "metadata.csv")


## Load weather data

In [4]:
# Weather data
weather_path = "..\\data\\weather\\"
weather = pd.read_csv(weather_path + "weather.csv")

In [5]:
# Convert timestamp field from string into pd.datetime object
weather['timestamp'] = pd.to_datetime(weather['timestamp'])

# Add column indicating the year, month and dayOfTheWeek for that timestamp
weather['date'] = weather['timestamp'].dt.date
weather['month'] = weather['timestamp'].dt.month

In [6]:
weather.head()

,timestamp,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed,date,month
0,2016-01-01 00:00:00,Panther,19.4,NaN,19.4,0.0,NaN,NaN,0.0,0.0,2016-01-01,1
1,2016-01-01 01:00:00,Panther,21.1,6.0,21.1,-1.0,NaN,1019.4,0.0,0.0,2016-01-01,1
2,2016-01-01 02:00:00,Panther,21.1,NaN,21.1,0.0,NaN,1018.8,210.0,1.5,2016-01-01,1
3,2016-01-01 03:00:00,Panther,20.6,NaN,20.0,0.0,NaN,1018.1,0.0,0.0,2016-01-01,1
4,2016-01-01 04:00:00,Panther,21.1,NaN,20.6,0.0,NaN,1019.0,290.0,1.5,2016-01-01,1


## Load raw dataset

In [7]:
raw_meters_path = "..\\data\\meters\\raw\\"

# files in directory
files = glob(raw_meters_path + "*.csv")

In [8]:

dfs = [] # empty list of the dataframes to create
for file in files: # for each file in directory
    meter_type = file.split("\\")[4].split(".")[0] # meter_type to rename the value feature
    meter = pd.read_csv(file) # load the dataset
    meter = pd.melt(meter, id_vars = "timestamp", var_name = "building_id", value_name = "meter_reading") # melt dataset
    meter["meter"] = str(meter_type) # adds column with the meter type
    dfs.append(meter) # append to list
raw_data = pd.concat(dfs, axis=0, ignore_index=True) # concatenate all meter
del(dfs, meter, file, files, meter_type)

raw_data.head()


,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Panther_office_Clementine,NaN,chilledwater
1,2016-01-01 01:00:00,Panther_office_Clementine,NaN,chilledwater
2,2016-01-01 02:00:00,Panther_office_Clementine,NaN,chilledwater
3,2016-01-01 03:00:00,Panther_office_Clementine,NaN,chilledwater
4,2016-01-01 04:00:00,Panther_office_Clementine,NaN,chilledwater


## Load cleaned dataset


In [9]:
cleaned_meters_path = "..\\data\\meters\\cleaned\\"

# files in directory
files = glob(cleaned_meters_path + "*.csv")


In [10]:
dfs = [] # empty list of the dataframes to create
for file in files: # for each file in directory
    meter_type = file.split("\\")[4].split(".")[0] # meter_type to rename the value feature
    meter = pd.read_csv(file) # load the dataset
    meter = pd.melt(meter, id_vars = "timestamp", var_name = "building_id", value_name = "meter_reading") # melt dataset
    meter["meter"] = str(meter_type) # adds column with the meter type
    dfs.append(meter) # append to list
complete_data_cleaned = pd.concat(dfs, axis=0, ignore_index=True) # concatenate all meter
del(dfs, meter, file, files, meter_type)

complete_data_cleaned.head()


,timestamp,building_id,meter_reading,meter
0,2016-01-01 00:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
1,2016-01-01 01:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
2,2016-01-01 02:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
3,2016-01-01 03:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned
4,2016-01-01 04:00:00,Panther_office_Clementine,NaN,chilledwater_cleaned


In [11]:
# Note this cell might take some time to finish

# Convert timestamp field from string into pd.datetime object
complete_data_cleaned['timestamp'] = pd.to_datetime(complete_data_cleaned['timestamp'])

# Add column indicating the year, month and dayOfTheWeek for that timestamp
complete_data_cleaned['date'] = complete_data_cleaned['timestamp'].dt.date
complete_data_cleaned['year'] = complete_data_cleaned['timestamp'].dt.year
complete_data_cleaned['month'] = complete_data_cleaned['timestamp'].dt.month
complete_data_cleaned['dayOfWeek'] = complete_data_cleaned['timestamp'].dt.dayofweek


In [12]:
meter_types = complete_data_cleaned['meter'].unique()
other_meter_types = meter_types.tolist().remove('electricity_cleaned')

# Load benchmarks


In [13]:
cleaned_meters_path = "..\\data\\"

# files in directory
files = glob(cleaned_meters_path + "*.csv")


benchmark = pd.read_csv(files[0]) # load the dataset

benchmark

,name,building_id,RMSE,MAE,horizon
0,Bear_utility_Sidney,utility,1.157131,0.846614,hourly
1,Bear_utility_Sidney,utility,1.255013,0.862390,daily
2,Bear_utility_Sidney,utility,1.851878,1.167219,weekly
3,Cockatoo_religion_Diedre,religion,1.475301,1.018945,hourly
4,Cockatoo_religion_Diedre,religion,2.349360,1.820794,daily
5,Cockatoo_religion_Diedre,religion,2.833513,1.958076,weekly
6,Cockatoo_science_Rex,science,7.304536,5.529282,hourly
7,Cockatoo_science_Rex,science,10.882962,7.975783,daily
8,Cockatoo_science_Rex,science,12.667458,8.261340,weekly
9,Eagle_education_Teresa,education,8.286079,5.855556,hourly


In [14]:
buildingNames = benchmark['name'].unique()

# Specify Constants


In order to obtain some of the constants we need to load a representative dataframe of one building for one meter_type

In [15]:
# We load the first 
building_name = buildingNames[0]
representative_df = complete_data_cleaned.loc[(complete_data_cleaned['building_id'] == building_name)
                                               & (complete_data_cleaned['meter'] == 'electricity_cleaned')]

In [16]:
TRAIN_TEST_SPLIT = 0.3
DATAPOINS_PER_BUILDING_AND_METER_TYPE = representative_df.shape[0]
SPLIT_INDEX = int(DATAPOINS_PER_BUILDING_AND_METER_TYPE * (1 - TRAIN_TEST_SPLIT))
SPLIT_TIMESTAMP =  representative_df.iloc[SPLIT_INDEX]['timestamp']
TRAIN_DIMENSIONS = 5
SAMPLE_SIZE = 2000
DATAPOINTS_ONE_WEEK = 24 * 7

In [17]:
representative_df[:SPLIT_INDEX]

,timestamp,building_id,meter_reading,meter,date,year,month,dayOfWeek
21561576,2016-01-01 00:00:00,Bear_utility_Sidney,53.0325,electricity_cleaned,2016-01-01,2016,1,4
21561577,2016-01-01 01:00:00,Bear_utility_Sidney,54.5325,electricity_cleaned,2016-01-01,2016,1,4
21561578,2016-01-01 02:00:00,Bear_utility_Sidney,55.1525,electricity_cleaned,2016-01-01,2016,1,4
21561579,2016-01-01 03:00:00,Bear_utility_Sidney,54.0900,electricity_cleaned,2016-01-01,2016,1,4
21561580,2016-01-01 04:00:00,Bear_utility_Sidney,53.9325,electricity_cleaned,2016-01-01,2016,1,4
...,...,...,...,...,...,...,...,...
21573851,2017-05-26 11:00:00,Bear_utility_Sidney,62.7850,electricity_cleaned,2017-05-26,2017,5,4
21573852,2017-05-26 12:00:00,Bear_utility_Sidney,61.9200,electricity_cleaned,2017-05-26,2017,5,4
21573853,2017-05-26 13:00:00,Bear_utility_Sidney,61.1675,electricity_cleaned,2017-05-26,2017,5,4
21573854,2017-05-26 14:00:00,Bear_utility_Sidney,57.9525,electricity_cleaned,2017-05-26,2017,5,4


# Gaussian Process Regression

## Helper Functions

In [18]:
"""
Normalizes the data to a Gaussian distribution using quantiles.
"""
def normalizeToGaussian(arr, mode="mean"):
    n = len(arr)
    perc = percentileofscore
    arr_ = arr.copy()[~np.isnan(arr)]
    out = np.zeros(n)
    for i in range(n):
        if not np.isnan(arr[i]):
            out[i] = norm.ppf(perc(arr_, arr[i], mode) / 100.)
        else:
            out[i] = np.nan
    return out


In [19]:
'''
Function that transforms the Gaussian normalisation back to percentiles of scores
'''
def gaussianToCdf(arr):
    n = len(arr)
    out = np.zeros(n)
    for i in range(n):
        if not np.isnan(arr[i]):
            out[i] = norm.cdf(arr[i])
        else:
            out[i] = np.nan
            
    out = out
    
    return out

In [20]:
def transformIntoOriginalRange(arr, original):

    
    original_min = np.min(original)
    original_max = np.max(original)
    original_mean = np.nanmean(original) 
    original_std = np.nanstd(original)

    # Calculate the range, mean, and standard deviation of the predictions
    arr_range = np.max(arr) - np.min(arr)
    arr_mean = np.mean(arr)
    arr_std = np.std(arr)

    # Rescale the predictions based on the original signal's range and statistics
    rescaled = (
        (arr - arr_mean) * (original_std / arr_std) + original_mean
    )

    # Adjust the rescaled predictions to fit within the original signal's range
    min_diff = original_min - np.min(rescaled)
    max_diff = np.max(rescaled) - original_max
    if min_diff > 0:
        rescaled -= min_diff
    elif max_diff > 0:
        rescaled -= max_diff
 
    return rescaled

In [21]:
'''
Replaces missing values in the dataframe with the mean of the month for all years from which we have data
@param dataframe
@returns numpy array
'''
def averageNaNs (df, field):
    mean_df = df.groupby(['month']).mean()
    if (len(mean_df.index) == 12) :
        averaged_mean = df[field].copy().fillna(
                                    df['month'].map({1: mean_df[field][1] , 2: mean_df[field][2], 3:mean_df[field][3],
                                                     4: mean_df[field][4] , 5: mean_df[field][5], 6:mean_df[field][6], 
                                                     7: mean_df[field][7] , 8: mean_df[field][8], 9:mean_df[field][9],
                                                     10: mean_df[field][10] , 11: mean_df[field][11], 12:mean_df[field][12]}))
    else :
        averaged_mean = df[field].copy().fillna(
                                    df['month'].map({5: mean_df[field][5], 6:mean_df[field][6], 
                                                     7: mean_df[field][7] , 8: mean_df[field][8], 9:mean_df[field][9],
                                                     10: mean_df[field][10] , 11: mean_df[field][11], 12:mean_df[field][12]}))
    averaged_numpy = averaged_mean.to_numpy()
    return averaged_numpy

In [22]:
'''
Train a Gaussian Process model based on the training data
@param two numpy arrays of same dimensionality
@returns numpy array with lenght NUMBER_OF_DIMENSIONS, containing the indices of the most correlated stations
'''
def trainGP (X_train, Y_train):
    
    # Shape of training data needs to be consistent 
    assert X_train.shape[0] == Y_train.shape[0]
    
    kernel = GPy.kern.RBF(input_dim=X_train.shape[1])
    model = GPy.models.GPRegression(X_train, Y_train, kernel)
    model.optimize(messages=True)
    model.optimize_restarts(num_restarts=5)
    
    return model

# So far unused helper functions

In [23]:
'''
Computation of Spearmas Rank Correlation
@param  target_df: a pandas.dataframe (target) for wich we want to calculate the correlation matrix 
        dataset: a list of pandas dataframes that build our base, as we check for the correlation between dataframe and each df in the truncated dataset
@returns spearmans_matrix: a numpy.ndarray that contains the correlation value as first entry in each row and the corresponding p-value as a second element in each row. 
                            the order of stations in the matrix is consistent to the order in the dataset:
'''
def spearmansCorrelation (target, possible_features): 
    y_target = target['meter_reading']
    spearmans_matrix = np.zeros((len(possible_features), 2))
    for index, feature in enumerate(possible_features): 
        if index % 100 == 0 : print(index)
        if ((target.shape[0] == feature.shape[0]) 
            and not ((feature['meter_reading'].isnull().sum()/feature.shape[0]) > 0.3)):
            correlation, _ = stats.spearmanr(y_target, feature['meter_reading'], nan_policy='omit')
            spearmans_matrix[index] = correlation, 0    

        else: 
            spearmans_matrix[index] = 0,100 
            continue
            
    return spearmans_matrix

### Load dataframe

### Transform into Gaussian

### Split into train and test sets

## Specify and Train Gaussian Process model

###### Plot the results given driectly from the GP

Note as we previously transformed the data to be normal distributed, the data we obtain here is as well transformed <br>
In order to reconstruct the original data we need to transform it back

In [24]:
'''
x = np.linspace(0, np.size(mean), np.size(mean))

plt.figure(figsize=(12,4))
plt.xlabel('Hours')
plt.ylabel('Energy_consumption')

plt.plot(x, mean, label='Prediction')
plt.plot(x, test_meter[0:200], label='Ground truth')
plt.fill_between(x, (mean - var).flatten(), (mean + var).flatten(), color='lightblue', alpha=0.5, label='Variance')
plt.legend()
plt.show()
'''

"\nx = np.linspace(0, np.size(mean), np.size(mean))\n\nplt.figure(figsize=(12,4))\nplt.xlabel('Hours')\nplt.ylabel('Energy_consumption')\n\nplt.plot(x, mean, label='Prediction')\nplt.plot(x, test_meter[0:200], label='Ground truth')\nplt.fill_between(x, (mean - var).flatten(), (mean + var).flatten(), color='lightblue', alpha=0.5, label='Variance')\nplt.legend()\nplt.show()\n"

## Convert the results back and compare them

In [25]:
'''
prediction_transformed = transformIntoOriginalRange(mean.flatten(), original_data_train)
variance_transformed = transformIntoOriginalRange(var.flatten(), original_data_train)
original_test_transformed = transformIntoOriginalRange(original_data_test[0:200], original_data_train)
'''

'\nprediction_transformed = transformIntoOriginalRange(mean.flatten(), original_data_train)\nvariance_transformed = transformIntoOriginalRange(var.flatten(), original_data_train)\noriginal_test_transformed = transformIntoOriginalRange(original_data_test[0:200], original_data_train)\n'

In [26]:
'''
x = np.linspace(0, np.size(prediction_transformed), np.size(prediction_transformed))

plt.figure(figsize=(12,4))
plt.xlabel('Hours')
plt.ylabel('Energy_consumption')

plt.plot(x, prediction_transformed, label='Prediction')
plt.plot(x, original_data_test[0:200], label='Ground truth')
plt.legend()
plt.show()
'''

"\nx = np.linspace(0, np.size(prediction_transformed), np.size(prediction_transformed))\n\nplt.figure(figsize=(12,4))\nplt.xlabel('Hours')\nplt.ylabel('Energy_consumption')\n\nplt.plot(x, prediction_transformed, label='Prediction')\nplt.plot(x, original_data_test[0:200], label='Ground truth')\nplt.legend()\nplt.show()\n"

# Error computation

In [27]:
def compute_error_metrics(y_target, prediction):
    assert (y_target.shape == prediction.shape)
    mse = mean_squared_error(y_target, prediction_transformed)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_target, prediction_transformed)

    
    return mse, rmse, mae


## Iteration over all buildings we want to predict

In [28]:
'''
For our given dataset we 
'''
def getDfsOfOtherMeterTypes(df): 
    dfs_meter_types = []
    meter_types = df['meter'].unique()

    for meterType in meter_types:
        if meterType == 'electricity_cleaned':
            continue
        
        dfs_meter_types.append(df.loc[df['meter'] == meterType]) 
        
    return dfs_meter_types

In [29]:
building_ids = complete_data_cleaned['building_id'].unique()
meter_types = complete_data_cleaned['meter'].unique()

In [30]:
# function not needed

def getAllMeterDfsExceptTarget(building_id): 
    dfs = []
    for id in building_ids:
        print(building_id)
        building = complete_data_cleaned.loc[complete_data_cleaned['building_id'] == id]
        for meter_type in building['meter'].unique(): 
            # We do not want to contain our target into this list of dataframes
            if ((id == building_id) and (meter_type == 'electricity_cleaned')):
                continue
            
            building_meter = building.loc[building['meter'] == meter_type]
            dfs.append(building_meter) 
        
    return dfs

In [1]:
# This cell might take some time to finish

# Group the DataFrame by 'building_id' and 'meter_type'
complete_data_grouped = complete_data_cleaned.groupby(['building_id', 'meter'])

# Initialize an empty list to store the smaller DataFrames
dfs = []
df_keys = []

# Iterate over the groups and create smaller DataFrames
for group_key, group in complete_data_grouped:
    df_keys.append(group_key)
    dfs.append(group.copy()) 

SyntaxError: invalid syntax (1608195383.py, line 4)

In [32]:
group_keys = np.array(df_keys)

In [33]:
# For given dataset (list of pd.Dataframes) we compute a subsample of those dataframes such that we obtain a list of dfs
# that do not fit the given building_name and are not of meter_type 'electricity_cleaned'
# Also we obtain the index/position of the dataframe that matches the building_id and is of type electricity_cleaned
def get_all_features(dataset, building_name):
    key_to_exclude = [building_name, 'electricity_cleaned']
    features = []
    index_of_df = np.nan
    for index, key in enumerate(group_keys):
        # as the key composes of two elements, the buildling_id and the meter_type we compare two arrays of strings, therefore we need the .all() 
        if(key == key_to_exclude).all():
            index_of_df = index
            continue
        features.append(dataset[index])
            
    return features, index_of_df

In [35]:
# Split a dataframe into train and test data according to a split_date
def split_df_to_train_and_test(df, split_date):
    df_train = df.loc[df['timestamp'] < split_date].copy().reset_index(drop=True) 
    df_test  = df.loc[df['timestamp'] >= split_date].copy().reset_index(drop=True)
    return df_train, df_test

In [36]:
dfs_train = []
dfs_test = []

for df in dfs: 
    
    temp_train_df, temp_test_df = split_df_to_train_and_test(df, SPLIT_TIMESTAMP)
    dfs_train.append(temp_train_df)
    dfs_test.append(temp_test_df)
        

In [37]:
# 
def sort_according_to_highest_correlation(spearmans_matrix):
    correlation_values = spearmans_matrix[:,0]
    clean_correlation_values = np.nan_to_num(correlation_values, 0)
    clean_abs_correlation_values = np.abs(clean_correlation_values)
    sorted_indices = np.argsort(clean_abs_correlation_values)[::-1]
    
    return sorted_indices

In [38]:
# For a dataset (list of pd.Dataframes) and a list of sorted indice
def transform_highest_correlated_dfs_to_training_data(list_of_dfs ,indices):
    transformed_features = []
    for ind in indices[:TRAIN_DIMENSIONS]:
        temp_df = list_of_dfs[ind]
        avg_temp_df = averageNaNs(temp_df, 'meter_reading')
        transformed_features.append(normalizeToGaussian(avg_temp_df))
    
    return np.column_stack((transformed_features))

        

In [39]:
def random_sampling(X_train, y_train, num_samples=SAMPLE_SIZE, random_seed=42):
    np.random.seed(random_seed)

    indices = np.random.choice(len(X_train), size=num_samples, replace=False)
    X_samples = X_train[indices]
    y_samples = y_train[indices]

    return X_samples, y_samples

# ToDos:

test if pearsons correlation is faster?
Check if it makes sense to compute once and for all this huge correlation matrix for every df with every other df?
One could compute this in an extra notebook, save it and then one could load it here and would not need to perform this super expensive computation for every dataframe


Not even needed to have all the correlation but only the ones of the 

In [ ]:

for name in buildingNames[0:1]: 
    print(name)
    
    # Get dataframe that matches our building_id
    df = complete_data_cleaned.loc[complete_data_cleaned['building_id']  == name]
    
    # For later reconstruction we split the electricity meter reading into train and test set
    y_train_df, y_test_df = split_df_to_train_and_test(df, SPLIT_TIMESTAMP)
    
    print("loading all other features ...")
    # Load all other features like water consumption, gas ....
    features_train, own_index = get_all_features(dfs_train, name)

    print("all other features loaded")
    
    # OPTIONAL
    # 1. Load the weather data for the location
    #siteName = name.split('_')[0]
    #weather = weather.loc[weather['site_id'] == siteName]
    # 2. Load the water data
    
    print("compute spearmans matrix ...")
    # Compute the spearmans correlation between all those features in order to pick the most relevant ones
    spearmans_matrix = spearmansCorrelation(y_train_df, features_train)
    print("spearmans matrix comptued", spearmans_matrix)

    
    # Sort according to highest correlation
    sorted_indices = sort_according_to_highest_correlation(spearmans_matrix)
    print("sorted indices: ", sorted_indices)
    
    
    print("transforming highest correlated dfs")
   
    # Bring data into format to fit as GP Regression input (normal distribution)
    X_train = transform_highest_correlated_dfs_to_training_data(dfs_train, sorted_indices)
    X_test = transform_highest_correlated_dfs_to_training_data(dfs_test, sorted_indices)

    print("X_train.shape", X_train.shape)
    print("X_test.shape", X_test.shape)

    Y_train = normalizeToGaussian(averageNaNs(y_train_df, 'meter_reading'))
    Y_test = normalizeToGaussian(averageNaNs(y_test_df, 'meter_reading'))
    
    
    print("Y_train.shape", Y_train.shape)
    print("Y_test.shape", Y_test.shape)

    sampled_X_train, sampled_Y_train = random_sampling(X_train, Y_train)
    
    print("sampled_X_train.shape", sampled_X_train.shape)
    print("sampled_Y_train.shape", sampled_Y_train.shape)
        
    # Randomly sample from the datapoints, as we cannot train on the whole dimensions
    
    print("Training model ....")
    # Train the model 
    model = trainGP (sampled_X_train, sampled_Y_train.reshape(-1,1))
    
    print("predicting ...")
    # Make predictions with the trained model for one week
    mean, var = model.predict(X_test[0:DATAPOINTS_ONE_WEEK])
    
    #Compute the error values for each timesoan hour, day, week
    

Bear_utility_Sidney
loading all other features ...
all other features loaded
compute spearmans matrix ...
0
100
200
300
400
500
600
700
800


 C:\Users\marku\anaconda3\lib\site-packages\scipy\stats\stats.py:4484: SpearmanRConstantInputWarning:An input array is constant; the correlation coefficient is not defined.


900
1000
1100
1200
1300
1400
1500
1600
1700
1800


In [ ]:
print("It finished!")
assert(False)



In [ ]:
print("transforming highest correlated dfs")
# Bring data into format to fit as GP Regression input (normal distribution)
X_train = transform_highest_correlated_dfs_to_training_data(dfs_train, sorted_indices)
X_test = transform_highest_correlated_dfs_to_training_data(dfs_test, sorted_indices)

print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape)


Y_train = normalizeToGaussian(averageNaNs(y_train_df, 'meter_reading'))
Y_test = normalizeToGaussian(averageNaNs(y_test_df, 'meter_reading'))


print("Y_train.shape", Y_train.shape)
print("Y_test.shape", Y_test.shape)

sampled_X_train, sampled_Y_train = random_sampling(X_train, Y_train)

print("sampled_X_train.shape", sampled_X_train.shape)
print("sampled_Y_train.shape", sampled_Y_train.shape)

# Randomly sample from the datapoints, as we cannot train on the whole dimensions

print("Training model ....")
# Train the model 
model = trainGP (sampled_X_train, sampled_Y_train.reshape(-1,1))

print("predicting ...")
# Make predictions with the trained model for one week
mean, var = model.predict(X_test[0:200])

#Compute the error values for each timesoan hour, day, week


In [ ]:
x = np.linspace(0, np.size(mean), np.size(mean))

plt.figure(figsize=(12,4))
plt.xlabel('Hours')
plt.ylabel('Energy_consumption')

plt.plot(x, mean, label='Prediction')
plt.plot(x, test_meter[0:200], label='Ground truth')
plt.fill_between(x, (mean - var).flatten(), (mean + var).flatten(), color='lightblue', alpha=0.5, label='Variance')
plt.legend()
plt.show()

In [ ]:
prediction_transformed = transformIntoOriginalRange(mean.flatten(), y_train_df['meter_reading'])
variance_transformed = transformIntoOriginalRange(var.flatten(), y_train_df['meter_reading'])
original_test_transformed = transformIntoOriginalRange(y_test_df['meter_reading'][0:200], y_train_df['meter_reading'])

In [ ]:
mse = mean_squared_error(y_test_df['meter_reading'][0:200], prediction_transformed)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_df['meter_reading'][0:200], prediction_transformed)

print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAE", mae)

In [ ]:
name